In [16]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from sampling import fetch_joined_data
df = await fetch_joined_data()
df = df.sort_values(['meeting_timestamp'])

In [18]:
from elastic_search import VectorSearchEngine
search_engine = VectorSearchEngine(device=0)
search_engine.create_index(df)


Using device: cuda:0
Creating field-specific indices...
Processing topic_name...


  0%|          | 0/400 [00:00<?, ?it/s]

Processing summary...


  0%|          | 0/400 [00:00<?, ?it/s]

Processing details...


  0%|          | 0/400 [00:00<?, ?it/s]

In [19]:
meeting_speaker_sets = df.groupby('meeting_id')['speaker_name'].agg(set)
meeting_speaker_sets = meeting_speaker_sets.apply(frozenset)
meeting_combinations = meeting_speaker_sets.value_counts()
print(meeting_combinations.head(20))

speaker_name
(Dmitriy Grankin, Olga Nemirovskaya)                                    33
(Dmitriy Grankin)                                                       32
(nan)                                                                   20
(Sergey Ryabenko, Dmitriy Grankin)                                      15
(nan (1), nan (0))                                                       6
(Andrey Pisankin, Dmitriy Grankin)                                       4
(Sergey Ryabenko, Dmitriy Grankin, Nikolay)                              3
(Dmitriy Grankin, Alex Shevliakov)                                       2
(nan (1), nan (0), nan)                                                  2
(Dmitriy Grankin, Matt Lewis)                                            2
(Ilia Semukhin, Dmitriy Grankin)                                         2
(Dmitry Grankin, Alex Shevliakov)                                        2
(Rick Tousseyn, Dmitriy Grankin)                                         2
(Andrey Pisa

In [20]:
team = ['Dmitry Grankin','Olga Nemirovskaya','Dmitriy Grankin','Sergey Ryabenko','Alex Shevliakov','Karine Stepanyan','Andrey Pisankin',
        'Okafor Ikenna','nan']

In [21]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [22]:
search_engine.get_search_hints("product ", max_hints=5)

['Product Hunt',
 'Product Hunter',
 'Product School',
 'product hunter',
 'Product Managers']

In [33]:
results = search_engine.search(
    query="competitors",
    k=20,
    min_similarity=0.49,
    exact_match_boost=0.3,  # Moderate boost for exact matches
    return_scores=True
)
results[['topic_name','speaker_name','summary','details','meeting_timestamp','similarity_score']]

,topic_name,speaker_name,summary,details,meeting_timestamp,similarity_score
1372,Analyze competitors' marketing strategies,Olga Nemirovskaya,Planned to analyze competitors' marketing strategies to inform future campaigns.,"Olga and Dmitriy discussed the importance of understanding competitors' successful strategies, particularly in TikTok and YouTube. This analysis will guide their content creation and marketing efforts. No deadline was set for this task.",2024-08-27 19:34:48.996,1.230171
2096,Identify key competitors for analysis,Dmitriy Grankin,"Dmitrii Bashkirov and Dmitriy Grankin discussed and identified key competitors to analyze, focusing on those with significant market presence.","Competitors selected for analysis include Fatom, TLDV, FireFiles, Otter, MidGeek, and SuperPower. The focus will be on understanding their marketing mechanics and user acquisition strategies.",2024-09-04 11:23:50.070,1.210878
2618,Identify user pain points with competitors,Dmitriy Grankin,Dmitriy discussed the need to understand user frustrations with competitors to better position their product.,Dmitriy highlighted that users often express dissatisfaction with bots and the quality of summaries provided by competitors. This insight can be leveraged to attract users looking for better alternatives.,2024-09-10 09:28:46.960,1.173120
1374,Create engaging video content based on competitors' success,Olga Nemirovskaya,Discussed creating video content inspired by successful competitors' strategies.,"Olga mentioned analyzing successful videos from competitors to replicate their success. This task is ongoing, with no specific deadline set.",2024-08-27 19:34:48.996,1.161271
2615,Analyze Google Ads usage among competitors,Olga Nemirovskaya,"Olga reviewed the Google Ads usage of competitors and found that most do not utilize it effectively, indicating a potential opportunity for differentiation.","Olga discovered that competitors like Fireflies, Notte, and Supernormal use Google Ads minimally, while others rely heavily on brand traffic and directories. This suggests a need to explore alternative marketing strategies that may be more cost-effective.",2024-09-10 09:28:46.960,1.128424
54,Differentiate product features from competitors,Dmitriy Grankin,Dmitriy highlighted the need to articulate the unique features of their product compared to competitors.,"They discussed the importance of identifying their product's advantages, such as no bots and multilingual support, to stand out in a competitive market.",2024-09-16 19:02:08.440,1.020887
281,Competitor analysis,Dmitriy Grankin,Dmitriy and Olga discussed identifying competitors and understanding their market positioning.,They need to analyze competitors in the note-taking and AI space to understand their offerings and how to differentiate their product.,2024-09-20 19:03:16.004,0.976851
2405,competitor SEO marketing,Dmitry Grankin,"Competitor SEO marketing is a subject of discussion in the meeting, focusing on analyzing competitors' strategies.","Dmitry brings up the topic of competitor SEO marketing, indicating a need to understand and possibly improve their own SEO strategies based on competitors' actions.",2024-09-06 14:32:06.780,0.947313
2091,Conduct a competitive analysis,Dmitriy Grankin,"Dmitriy Grankin emphasized the need to conduct a competitive analysis to understand how competitors acquire users, suggesting that this step is crucial before launching marketing efforts.","Dmitrii Bashkirov will assist in gathering data on competitors' user acquisition strategies, focusing on 3-6 key competitors in the B2B space. The analysis will include traffic sources and onboarding processes. The goal is to identify effective marketing mechanics used by competitors.",2024-09-04 11:23:50.070,0.925308
3131,Competitor Comparison Writing,Olga Nemirovskaya,Olga proposed writing comparisons of competitors related to the Google Meta feature to enhance engagement.,"The idea is to create content that aligns with the Google 

In [41]:
results = search_engine.search_by_speaker(
    "robert",
    k=50,
    min_similarity=0.49,  # Higher threshold for more exact matches
    return_scores=True
)
results

,speaker_name,topic_name,summary,details,meeting_timestamp,speaker_similarity
2548,Robert Hangu,Linux,Linux is an operating system that Robert uses for programming and development.,"Robert mentions using Linux and a Tiling Window Manager, indicating his preference for efficient and minimalistic tools.",2024-09-09 10:46:17.724,0.550814
2543,Robert Hangu,AI,"AI refers to artificial intelligence, a key topic in the meeting as both speakers discuss its applications in their work.",The conversation revolves around the integration of AI in tools like Vexa and its impact on productivity and efficiency.,2024-09-09 10:46:17.724,0.550814
2531,Robert Hangu,Discussion on the effectiveness of AI tools for summarizing meetings,"Robert shared his experience with Vexa and Fireflies, highlighting the importance of having a tool that allows for interaction with the transcript rather than just providing a static summary.","Robert mentioned that he previously used Fireflies but found its AI summaries lacking. He now uses Vexa to extract specific information from call transcripts, which he finds much more effective. He emphasized the need for tools that allow for open-ended questions to gain deeper insights.",2024-09-09 10:46:17.724,0.550814
2533,Robert Hangu,Robert's workflow using Vexa and ChatGPT,Robert outlined his workflow for using Vexa in conjunction with ChatGPT to enhance his note-taking and information extraction process.,"He described how he uses Vexa to transcribe calls and then prompts ChatGPT for specific insights, which he finds saves time and improves the quality of his notes. He has developed a system for copying relevant information into Google Docs for future reference.",2024-09-09 10:46:17.724,0.550814
2537,Robert Hangu,Robert Hangu,"Robert Hangu is a speaker in the meeting, providing feedback on the Vexa tool and sharing his experiences with similar tools.","Robert has used Vexa and Fireflies for meeting transcription and feedback, and he shares insights on their functionalities and user experience.",2024-09-09 10:46:17.724,0.550814
2538,Robert Hangu,Vexa,Vexa is a real-time assistant for meetings that helps users transcribe and summarize discussions.,"Robert Hangu provides positive feedback about Vexa, highlighting its clean interface and ability to generate useful summaries from meeting transcripts.",2024-09-09 10:46:17.724,0.550814
2539,Robert Hangu,Fireflies,Fireflies is a meeting transcription tool that Robert Hangu previously used before switching to Vexa.,Robert found Fireflies' AI summaries to be inadequate and prefers Vexa's more user-friendly interface and functionality.,2024-09-09 10:46:17.724,0.550814
2540,Robert Hangu,ChatGPT,ChatGPT is an AI language model that Robert Hangu uses to generate insights and summaries from meeting transcripts.,"Robert uses ChatGPT to refine the information extracted from transcripts, enhancing his workflow and productivity.",2024-09-09 10:46:17.724,0.550814
2541,Robert Hangu,Hyper SEO,Hyper SEO is a topic discussed in the context of trial offers during the meeting.,"Robert inquired about the trial offer related to Hyper SEO during his conversation with Vexa, seeking specific details.",2024-09-09 10:46:17.724,0.550814
2542,Robert Hangu,Next Jobs,Next Jobs is an AI-powered job board launched by Robert Hangu.,"Robert is working on this project alongside his consulting business, focusing on integrating generative AI into various applications.",2024-09-09 10:46:17.724,0.550814
